In [1]:
import pandas as pd
import numpy as np
import re
import datetime
pd.options.display.max_columns = None

Seguiremos trabajando con mismos csv que en el pair anterior, el de los empleados que unimos en el pair de unión de datasets y el de los tickets de todos los años. Para ello, lo primero que tendremos que hacer es cargar estos dos csv.

In [2]:
df = pd.read_csv('historico_tickets.csv', index_col = 0)
df.sample(3)

,id_ticket,fecha,id_empleado,id_agente,categoría,tipo,severidad,prioridad,días_resolución,satisfacción,semestre,año
8321,KWLTER-5842809069,2017-03-15,1526,30,Software,Solicitud,2 - Normal,1 - Baja,8,1,1 semestre,2017
8286,KHLEER-0443514179,2019-02-18,437,11,Software,Problema,2 - Normal,3 - Alta,1,2,1 semestre,2019
3138,GWLEER-0442613746,2016-08-31,204,47,Software,Problema,2 - Normal,1 - Baja,4,5,2º semestre,2016


1- Cread una columna nueva y una función que nos de el mes en el que se solicitó el servicio. Tened en cuenta que todas las filas tienen la misma estructura. A la hora de escribir la función debemos tener en cuenta que puede que no haya la información de mes en la columna de la fecha. En ese caso devolved un nulo (NaN).

📌 Nota La fecha tendrá que ir formato de string como "Enero", "Febrero", etc.

💡 Pista 💡Podréis usar regex, pero no es estrictamente necesario.

In [3]:
df['fecha'] = df['fecha'].apply(pd.to_datetime)

In [4]:
def sacar_mes(f):
    return f.strftime("%B")

In [5]:
df['mes'] = df["fecha"].apply(sacar_mes)

In [6]:
df.sample(3)

,id_ticket,fecha,id_empleado,id_agente,categoría,tipo,severidad,prioridad,días_resolución,satisfacción,semestre,año,mes
6785,GMLTNR-9044001669,2020-06-19,1686,26,Acceso/Login,Solicitud,2 - Normal,2 - Media,0,5,1 semestre,2020,June
7782,GWLTER-3443966748,2020-05-15,1634,27,Software,Solicitud,2 - Normal,1 - Baja,4,1,1 semestre,2020,May
5633,KHLTET-5542636532,2016-09-23,103,5,Software,Solicitud,2 - Normal,3 - Alta,5,4,2º semestre,2016,September


2- En la tabla de empleados, tenemos los nombres y los apellidos de los empleados, pero en columnas separadas. En los ejercicios de pair de la lección del groupby vimos que si quisieramos agrupar por los nombres, esto se nos puede complicar. Ahora nos piden que creemos una columna donde tengamos el nombre y el apellido juntos.

📌 NOTA La columna nueva debe tener el nombre y apellido con la primera letra en mayúsculas, por ejemplo "Jesús Villa".

📌 NOTA Una vez hallamos creado la columna nueva, eliminad las columnas de nombre y apellido originales.

In [29]:
dfe = pd.read_csv('empleados_puestos.csv', index_col = 0)
dfe.sample(3)

,Turno,ID Empleado,Apellido,Nombre,ID Puesto,Email,Puesto,Departamento,Tipo Puesto
373,NaN,178,VALDEZ,FLORA,15-24100,flora.valdez@tractchun.com,Supervisor de Producción,PRODUCCION,Mandos Medios
167,NaN,269,Betancur,Jeisson,15-24100,jeisson.betancur@tractchun.com,Supervisor de Producción,PRODUCCION,Mandos Medios
1239,NaN,851,CASTELLANOS,ARACELI,80-22210,araceli.castellanos@tractchun.com,Supervisor de Mantenimiento,MANTENIMIENTO,Mandos Medios


In [31]:
def nombre_completo(nombre, apellido):
    try:
        nc = nombre.title() + ' ' + apellido.title()
        return nc
    except:
        return np.nan

In [36]:
dfe['Nom_Completo'] = dfe.apply(lambda dataf: nombre_completo(dataf['Nombre'], dataf['Apellido']), axis = 1)

In [41]:
dfe.drop('Apellido', axis = 1, inplace = True)
dfe.drop('Nombre', axis = 1, inplace = True)

In [42]:
dfe.sample(3)

,Turno,ID Empleado,ID Puesto,Email,Puesto,Departamento,Tipo Puesto,Nom_Completo
1756,NaN,1893,10-32100,eduardo.bejarano@tractchun.com,Supervisor de Almacén,MATERIALES,Mandos Medios,Eduardo Bejarano
1081,NaN,951,45-11000,guadalupe.machado@tractchun.com,Supervisor de Aseguramiento de Calidad,INGENIERIA DE CALIDAD,Mandos Medios,Guadalupe Machado
1396,NaN,311,15-24111,julio.chomina@tractchun.com,Operador de Producción,PRODUCCION,Operativo,Julio Chomina


3- En el pair programming desde a dirección nos pedían que agruparamos los datos en base a la columna de "categoría", puede que en aquel momento nos dieramos cuenta que había varías categorías que eran igual (en concreto "Acceso Login") pero que en algunos casos tenía unos símbolos raros (&, %, -). 

Es el momento de limpiar esta columna. El objetivo, reemplazar todos estos símbolos raros por "/", para conseguir homogeneizar los valores de esta columna. Para ellos deberemos crear una función.

In [43]:
df.sample(2)

,id_ticket,fecha,id_empleado,id_agente,categoría,tipo,severidad,prioridad,días_resolución,satisfacción,semestre,año,mes
11362,SMLTSR-9342890996,2017-06-04,1909,49,Sistema,Solicitud,2 - Normal,2 - Media,12,5,1 semestre,2017,June
8720,SHLENT-5042522349,2016-06-01,104,3,Acceso&%Login,Problema,2 - Normal,3 - Alta,0,5,1 semestre,2016,June


In [46]:
categorías = df['categoría'].unique()
categorías

array(['Acceso&%Login', 'Sistema', 'Software', 'Hardware',
       'Acceso%&Login', 'Acceso-&Login', 'Acceso99Login', 'Acceso/Login'],
      dtype=object)

In [49]:
def limpiar_cat(c):
    if c == 'Acceso&%Login' or c == 'Acceso%&Login' or c == 'Acceso-&Login' or c == 'Acceso99Login':
        return 'Acceso/Login'
    else:
        return c

In [50]:
df['categoría'] = df['categoría'].apply(limpiar_cat)

In [52]:
df['categoría'].unique()

array(['Acceso/Login', 'Sistema', 'Software', 'Hardware'], dtype=object)

4- Una vez que hayáis terminado eliminad las columnas originales conservando las nuevas.

In [54]:
df.sample(4)

,id_ticket,fecha,id_empleado,id_agente,categoría,tipo,severidad,prioridad,días_resolución,satisfacción,semestre,año,mes
23022,SWLTSR-9244129764,2020-10-25,456,7,Sistema,Solicitud,2 - Normal,1 - Baja,12,5,2º semestre,2020,October
415,GDLEST-8343771301,2019-11-02,1500,43,Sistema,Problema,2 - Normal,0 - Sin Asignar,3,5,2º semestre,2019,November
7880,KMLTSR-3942904157,2017-06-18,565,41,Sistema,Solicitud,2 - Normal,2 - Media,9,4,1 semestre,2017,June
14244,KMLENT-9044110493,2020-10-06,329,14,Acceso/Login,Problema,2 - Normal,2 - Media,0,5,2º semestre,2020,October


5- Guardad el dataframe en un csv que usaremos más adelante

In [55]:
df.to_csv("datos/tickets_final.csv")
dfe.to_csv("datos/empleados_final.csv")